<a href="https://colab.research.google.com/github/neerajgoyal12/electoral_bonds/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [415]:
import pandas as pd
from pathlib import Path
import requests

# Download helper functions from Learn PyTorch repo (if not already downloaded)
if Path("helper_functions.py").is_file():
  print("helper_functions.py already exists, skipping download")
else:
  print("Downloading helper_functions.py")
  request = requests.get("https://raw.githubusercontent.com/neerajgoyal12/electoral_bonds/main/helper_functions.py")
  with open("helper_functions.py", "wb") as f:
    f.write(request.content)

if Path("ec_data.csv").is_file():
  print("ec_data.csv already exists, skipping download")
else:
  print("Downloading ec_data.csv")
  request = requests.get("https://raw.githubusercontent.com/neerajgoyal12/electoral_bonds/main/data/ec_data.csv")
  with open("ec_data.csv", "wb") as f:
    f.write(request.content)


helper_functions.py already exists, skipping download
ec_data.csv already exists, skipping download


In [416]:
df = pd.read_csv('ec_data.csv')

In [417]:
df.head(5)

,Date of Purchase,Purchaser Name,Denomination,Unnamed: 3
0,12/Apr/2019,A B C INDIA LIMITED,100000,NaN
1,12/Apr/2019,A B C INDIA LIMITED,100000,NaN
2,12/Apr/2019,A B C INDIA LIMITED,1000000,NaN
3,12/Apr/2019,A B C INDIA LIMITED,1000000,NaN
4,12/Apr/2019,A B C INDIA LIMITED,100000,NaN


In [418]:
df.tail(5)

,Date of Purchase,Purchaser Name,Denomination,Unnamed: 3
18866,11/Jan/2024,WESTWELL GASES PRIVATE LIMITED,10000000,NaN
18867,11/Jan/2024,WESTWELL GASES PRIVATE LIMITED,10000000,NaN
18868,11/Jan/2024,WESTWELL GASES PRIVATE LIMITED,10000000,NaN
18869,11/Jan/2024,WESTWELL GASES PRIVATE LIMITED,10000000,NaN
18870,11/Jan/2024,WESTWELL GASES PRIVATE LIMITED,10000000,NaN


In [419]:
#dropping NaN column
df = df.dropna(axis = 1, how = 'all')
df

,Date of Purchase,Purchaser Name,Denomination
0,12/Apr/2019,A B C INDIA LIMITED,100000
1,12/Apr/2019,A B C INDIA LIMITED,100000
2,12/Apr/2019,A B C INDIA LIMITED,1000000
3,12/Apr/2019,A B C INDIA LIMITED,1000000
4,12/Apr/2019,A B C INDIA LIMITED,100000
...,...,...,...
18866,11/Jan/2024,WESTWELL GASES PRIVATE LIMITED,10000000
18867,11/Jan/2024,WESTWELL GASES PRIVATE LIMITED,10000000
18868,11/Jan/2024,WESTWELL GASES PRIVATE LIMITED,10000000
18869,11/Jan/2024,WESTWELL GASES PRIVATE LIMITED,10000000


In [420]:
# For future easy ness
dstr = 'Date of Purchase'
pnstr = 'Purchaser Name'
denostr = 'Denomination'

In [421]:
df.infer_objects()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18871 entries, 0 to 18870
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Date of Purchase  18871 non-null  object
 1   Purchaser Name    18871 non-null  object
 2   Denomination      18871 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 442.4+ KB


In [422]:
# Converting dtypes for later convenience
df[dstr] = pd.to_datetime(df[dstr])
df[pnstr] = df[pnstr].astype(str)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18871 entries, 0 to 18870
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date of Purchase  18871 non-null  datetime64[ns]
 1   Purchaser Name    18871 non-null  object        
 2   Denomination      18871 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 442.4+ KB


In [423]:
# prompt: in df clean pnstr

import re

def clean_string(string):
  # Remove leading and trailing whitespace
  string = string.strip()
  # Convert to lowercase
  string = string.lower()
  # Remove special characters
  string = re.sub(r'[^a-zA-Z0-9\s]', '', string)
  # Remove multiple spaces
  string = re.sub(r'\s+', ' ', string)
  return string

df[pnstr] = df[pnstr].apply(clean_string)


In [424]:
# prompt: in df pnstr replace 'megha engineering infrastructures limited'  with ' 'megha engineering and infrastructures limited'

df[pnstr] = df[pnstr].apply(lambda x: x.replace('megha engineering infrastructures limited', 'megha engineering and infrastructures limited'))


In [425]:
# prompt: in df pnstr replace '.' with ' '

df[pnstr] = df[pnstr].apply(lambda x: x.replace('.', ' '))


In [426]:
grouped_df = df.groupby(pnstr, sort=False).sum(numeric_only=True).copy()
grouped_df.reset_index(inplace=True)
grouped_df.sort_values(by=denostr, ascending=False, inplace=True)


In [427]:
!pip install jellyfish


In [428]:
def combine_similar_names(df, threshold=0.9):
  """
  Combines rows in a DataFrame where the 'Purchaser Name' column values are similar by a specified threshold.

  Args:
      df: The DataFrame to process.
      threshold: The similarity threshold (0 to 1) for combining names.

  Returns:
      A new DataFrame with combined rows.
  """

  combined_df = df.copy()
  for i in range(len(df)):
    for j in range(i + 1, len(df)):
      name1 = df.loc[i, 'Purchaser Name']
      name2 = df.loc[j, 'Purchaser Name']
      names1 = name1.split()
      names2 = name2.split()
      first_similarity = jaro_similarity(names1[0], names2[0])
      # print(f"i: {names1[0]} j: {j} {names2[0]} | {first_similarity}")
      if first_similarity < 0.97:
        continue
      else:
        similarity = jaro_similarity(name1, name2)
        if similarity >= threshold:
          combined_df.loc[i, 'Denomination'] += df.loc[j, 'Denomination']
          # print(f"i: {i} j: {j} {name1} | {name2} | {similarity}")
          combined_df.drop(index=j, inplace=True)
          combined_df.reset_index(inplace=True, drop=True)
        else:
          print(f"i: {i} j: {j} {name1} | {name2} | {similarity}")
  return combined_df


In [429]:
# droppinf donners below 1 cr
above_1_cr_grouped_df = grouped_df[grouped_df[denostr] > 10000000].copy()
above_1_cr_grouped_df.tail()


,Purchaser Name,Denomination
1175,harijan hiribai,10768000
1183,rathod lakhiben,10768000
1172,bhachiben khamubhai manvar,10768000
473,ranisati mercantiles pvt ltd,10500000
65,raipur botling company,10001000


In [430]:
print(f"Highest Donation : {above_1_cr_grouped_df[denostr].max()}")
print(f"Lowest Donation : {above_1_cr_grouped_df[denostr].min()}")
above_1_cr_grouped_df[denostr] = above_1_cr_grouped_df[denostr] / 10000000

Highest Donation : 12080000000
Lowest Donation : 10001000


In [431]:
above_1_cr_grouped_reseted_df = above_1_cr_grouped_df.reset_index(drop=True, inplace=False).copy()
print(f"Above 1 cr total: {len(above_1_cr_grouped_reseted_df)}")
above_1_cr_grouped_combined_df = combine_similar_names(above_1_cr_grouped_reseted_df, threshold=0.90)
print(f"After combining similar names: {len(above_1_cr_grouped_combined_df)}")

Above 1 cr total: 671
i: 4 j: 621 vedanta limited | vedanta electricals private limited | 0.6947496947496948
i: 9 j: 51 bharti airtel limited | bharti telemedia limited | 0.8285714285714286
i: 9 j: 134 bharti airtel limited | bharti airtel limitedairtel curre nt acgco | 0.8333333333333334
i: 12 j: 81 dlf commercial developers limited | dlf luxury homes limited | 0.6663343965975544
i: 12 j: 135 dlf commercial developers limited | dlf garden city indore pvt ltd | 0.6676767676767676
i: 14 j: 67 jindal steel and power limited | jindal stainless ltd | 0.7592592592592592
i: 14 j: 69 jindal steel and power limited | jindal saw limited | 0.7740740740740741
i: 14 j: 191 jindal steel and power limited | jindal house | 0.688888888888889
i: 14 j: 444 jindal steel and power limited | jindal poly films limited | 0.7222222222222223
i: 17 j: 505 avees trading finance pvt ltd | avees trading and finance pvt ltd | 0.8791361894810171
i: 23 j: 45 torrent power limited | torrent pharmaceuticals ltd | 0.754

KeyError: '[636] not found in axis'

In [ ]:
# prompt: convert denomination to indian rupee in crore

above_1_cr_grouped_df[denostr] = above_1_cr_grouped_df[denostr] / 10000000
print(f"Total unique donors above 1cr: {len(above_1_cr_grouped_combined_df)}")

In [ ]:
top_50_donors = above_1_cr_grouped_combined_df.head(50).copy()
top_50_donors.head()

In [ ]:
top_50_donors_reseted = top_50_donors.reset_index(drop=True, inplace=False).copy()
top_50_donors_reseted.head()


In [ ]:
# print(f"Before combining : {len(top_50_donors_reseted)}")
# top_50_donors_combined = combine_similar_names(top_50_donors_reseted, threshold=0.90)
# print(f"After combining : {len(top_50_donors_combined)}")



In [ ]:
ax = top_50_donors_reseted.plot.barh(x=pnstr, y=denostr)

# Add labels for the denominations on the right side of the bars
for i, bar in enumerate(ax.patches):
    ax.text(bar.get_width() + 50, bar.get_y() + bar.get_height() / 2,
            str(int(bar.get_width())), ha='center', va='center')

# Invert the y-axis to show the highest values at the top
plt.gca().invert_yaxis()

# Add a title and show the plot
plt.title("Top 50 Donors (in Crores)")

import os

# Create the directory if it doesn't exist
if not os.path.exists('data'):
  os.makedirs('data')

# Save the figure
plt.savefig('data/top_50_donors_in_crore.png', bbox_inches='tight')
plt.show()